In [190]:
from bs4 import BeautifulSoup
import numpy as np
import requests
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# Part 1

We are given the base URL which we denote by url:
http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247
We are going to use requests and BeautifulSoup libraries to get the data from the webpage. We observe that the filters that we saw on the page are in the frame, so we parse the url to get the needed new url.

In [46]:
url="http://isa.epfl.ch/imoniteur_ISAP/%21gedpublicreports.htm?ww_i_reportmodel=133685247"

r = requests.get(url)
data = r.text
soup = BeautifulSoup(data,"lxml")

gedPublicReports = soup.find_all("frame")[1].extract().get("src")
urlPage = url.split("%21")[0] + gedPublicReports

Next we want to get a new page (with filters), from where would get the necessary parameters for the url with data (observed through postman). After checking it's format, we notice that we are interested in fields 'input' and 'select', that's where the filters are.

In [53]:
r1 = requests.get(urlPage)
fullData = r1.text
soup1 = BeautifulSoup(fullData,"lxml")

inputFields = soup1.find_all('input')
print(inputFields) # just for information on how we can observe the html data

# since we need to choose html or xls format, we delete the other one we don't need (here, html)
del inputFields[3]
attr = inputFields[5]
del inputFields[5] # this field will be used after to assign department (parameters as viewed from Postman)

# we do similar operations with 'select' tag to get the remaining parameters
selectFields = soup1.find_all('select')
attrName = attr.get("name")
attrValue = selectFields[0].extract().findAll('option')[9].text
attName=selectFields[0].extract().get("name")
attValue=selectFields[0].extract().findAll('option')[9].get('value')

[<input name="ww_b_list" type="hidden" value="1"/>, <input name="ww_i_reportmodel" type="hidden" value="133685247"/>, <input name="ww_c_langue" type="hidden" value=""/>, <input checked="" name="ww_i_reportModelXsl" type="radio" value="133685270"/>, <input name="ww_i_reportModelXsl" type="radio" value="133685271"/>, <input name="zz_x_UNITE_ACAD" type="hidden" value=""/>, <input name="zz_x_PERIODE_ACAD" type="hidden" value=""/>, <input name="zz_x_PERIODE_PEDAGO" type="hidden" value=""/>, <input name="zz_x_HIVERETE" type="hidden" value=""/>, <input name="dummy" type="submit" value="ok"/>]


Now we need to create a url with the found parameters. For that we define a function

In [54]:
def createUrl(url, l):
    for el in l:
        name = el.extract().get("name")
        value = el.extract().get("value")
        url = url + '&' + name + '=' + value
    return url

And apply it to get the final URL:

In [55]:
x = createUrl(urlPage,inputFields)
finalUrl = x + "&" + attrName + "=" + attrValue + "&" + attName + "=" + attValue

At this point our finalUrl will bring us to the page with necessary filters selected, one step before loading the actual data (when we click on 'Tous'). To load the data, we get the url that is loaded when the button is clicked.

In [56]:
from pyparsing import QuotedString
from itertools import chain

r2  = requests.get(finalUrl)
data2 = r2.text
soup2 = BeautifulSoup(data2,"lxml")

gps = QuotedString("'", unquoteResults = True).searchString(soup2('a', onclick = True)[0]['onclick'])[0][0]

Thanks to this, we are able to get the last parameter and get the data.

In [57]:
finalDataUrl = (finalUrl + "&" + gps).replace("filter","html")
r3  = requests.get(finalDataUrl)
data3 = r3.text
soup3 = BeautifulSoup(data3,"lxml")

Now we get the table from html and we want to load it in a dataframe.

In [144]:
bs4Text = soup3.find_all('table')[0]
dataFrame = pd.read_html(str(bs4Text))

After looking at the data, we observe that the first row is just additional information, so we delete that row.
We want also to write the information related to 'Semester' in a separate column, so we start by creating an empty one.

In [387]:
x = dataFrame[0].drop(0)
x["semester"] = numpy.nan

Now we fill in the column 'Semester' with the appropriate semester and drop the info rows.

In [388]:
#save semester in a column
semester="Unknown"
for index, row in x.iterrows():
    x.loc[index,"semester"] = semester 
    if row.drop([0]).isnull().all():
        semester = row[0]
        x.drop(index, inplace=True)

Next, we want to make the first row of the dataframe as an index and remove all their occurences inside the dataframe. We rename the column names to have the right names.

In [390]:
x.columns = x.iloc[0]
x = x[x[x.columns[1]].str.contains("Nom Prénom") == False]
x = x.rename(columns = {x.columns[len(x.columns)-1]:'Semester'})

Drop all the columns that have all values as NA.

In [391]:
df = x.dropna(axis = 1, how = 'all')

In [392]:
#y.head(5)

Keep just the years starting from 2007

In [393]:
years = map(str, np.arange(1980, 2007))
y = df[df[df.columns[len(df.columns)-1]].str.contains('|'.join(years)) == False]

Finally, let's reset the index and create a new one consisting of 'Sciper' and 'Semester'.

In [394]:
z = y.reset_index().drop("index",axis=1)
dfF2 = z.set_index(['No Sciper','Semester'])
dfF2.head(5)

,2,Civilité,Nom Prénom,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange
No Sciper,Semester,,,,,,,,
179749,"Echange IN, 2007-2008, Semestre automne (17 ét.)",Monsieur,Bouaziz Sofien,NaN,NaN,NaN,Présent,Erasmus,Ecole Supérieure d'Ingénieurs en Electronique ...
180104,"Echange IN, 2007-2008, Semestre automne (17 ét.)",Monsieur,Charles Christian,NaN,NaN,NaN,Présent,Erasmus,Ecole Supérieure de Chimie Physique Electroniq...
181031,"Echange IN, 2007-2008, Semestre automne (17 ét.)",Monsieur,Dagand Pierre-Evariste,NaN,NaN,NaN,Présent,Erasmus,Ecole Normale Supérieure de Cachan
179911,"Echange IN, 2007-2008, Semestre automne (17 ét.)",Monsieur,Grataloup Olivier,NaN,NaN,NaN,Présent,Erasmus,Ecole Supérieure de Chimie Physique Electroniq...
179934,"Echange IN, 2007-2008, Semestre automne (17 ét.)",Monsieur,Grignard Arnaud,NaN,NaN,NaN,Présent,Erasmus,Ecole Supérieure de Chimie Physique Electroniq...


Now we want to replace the semester column with the corresponding year. We will use this information later to compute (accurately) the number of months the student was at EPFL. 

In [395]:
import re
def extractYear(x):
    pattern1 = re.compile('(20..)-.*')
    pattern2 = re.compile('-(20..).*')
    if(x.find("Bachelor semestre 1") >= 0):
        year = re.findall(pattern1, x)
    elif((x.find("semestre 6") >= 0)):
        year = re.findall(pattern2, x)
    elif((x.find("semestre 5") >= 0)):
        year = re.findall(pattern1, x)
    else: year = ['0']
    return int(year[0])

# example of usage
x = "gqffeq semestre 5 2004-2005"
y = extractYear(x)
print(y)

2004


Here we want to keep only the students that have entry for Bachelor 1 and Bachelor 6 and get their scipers.

In [396]:
bachl_1 = dfF2.iloc[dfF2.index.get_level_values(1).str.contains('Bachelor semestre 1')].index.get_level_values(0)
bachl_1_6 = dfF2.iloc[dfF2.index.get_level_values(1).str.contains('Bachelor semestre 6') & dfF2.index.get_level_values(0).isin(bachl_1) ]
scipers_1_6 = bachl_1_6.index.get_level_values(0)

# filter out the students that are not in the list of scipers
filtered = dfF2.iloc[dfF2.index.get_level_values(0).isin(scipers_1_6) & dfF2.index.get_level_values(1).str.contains('Bachelor')]
filtered.reset_index(level = 1, inplace=True)

Function to compute the number of months depending on the start and finish years 

In [397]:
def computeMonths(x):
    lastMonths = [x[-2], x[-1]]
    months = (x[-1] - x[0])*12
    if(x[-2] ==  x[-1]):
        months += 6 
    return months

 We will apply the extractYear function to get the starting year and the finishing year. Then we will use this information to calculate the number of months (which is displayed below).

In [398]:
filtered['Semester'] = filtered['Semester'].map(lambda x : extractYear(x))
filtered = filtered.loc[filtered['Semester']!= 0]

filtered = filtered.set_index([filtered.index.get_level_values(0), 'Semester'])
filtered_list = filtered.groupby(filtered.index.get_level_values(0)).apply(lambda x: list(x.index.get_level_values(1)))
months = pd.DataFrame(filtered_list)
months = months.rename(columns = {0:'months'})
months = months['months'].apply(computeMonths)
months = pd.DataFrame(months)
months.head()

,months
No Sciper,
147008,36
169569,36
169731,48
169795,54
171195,36


Now we want to merge the previous dataframe with our original (filtered) one. The head of the final dataframe is displayed below.

In [399]:
toMerge = z.set_index(['No Sciper'])

#drop duplicates
toMerge = toMerge.groupby(toMerge.index).first()

#get other columns
fullData=pd.concat([months,toMerge], axis=1,join='inner')
fullData.head()

,months,Civilité,Nom Prénom,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange,Semester
No Sciper,,,,,,,,,,
147008,36,Monsieur,Good Xavier,"Signals, Images and Interfaces",NaN,NaN,Présent,Bilatéral,Carnegie Mellon University Pittsburgh,"Informatique, 2008-2009, Bachelor semestre 1 ..."
169569,36,Monsieur,Arévalo Christian,"Signals, Images and Interfaces",NaN,NaN,Présent,NaN,NaN,"Informatique, 2007-2008, Bachelor semestre 1 ..."
169731,48,Monsieur,Knecht Mathieu,NaN,NaN,NaN,Présent,NaN,NaN,"Informatique, 2007-2008, Bachelor semestre 1 ..."
169795,54,Monsieur,Scheiben Pascal,NaN,NaN,NaN,Présent,NaN,NaN,"Informatique, 2007-2008, Bachelor semestre 1 ..."
171195,36,Monsieur,Richter Arnaud,NaN,NaN,NaN,Présent,NaN,NaN,"Informatique, 2007-2008, Bachelor semestre 1 ..."


Finally, we can calculate the average number of months for female and male students

In [400]:
avgMonths = fullData.groupby(fullData.columns.values.tolist()[1]).mean()
avgMonths

,months
Civilité,
Madame,40.758621
Monsieur,42.880435


We can see that the average number of months to finish the bachelors is slightly more for men than for women. This difference could be due to the fact that the female sample is rather small. However, we can still run the two-sided two sample t-test to test the significance of the data. Hypothesis: Female students finish bachelor in shorter or longer period of time than male students => Two-sided t-test against the null hypothesis that the means for two samples are equal.
So, we prepare the data for the test.

In [401]:
men = fullData.loc[fullData[fullData.columns.values.tolist()[1]] == "Monsieur"]["months"]
women = fullData.loc[fullData[fullData.columns.values.tolist()[1]] == "Madame"]["months"]

In [402]:
men.shape, women.shape

((368,), (29,))

In [403]:
# additional imports for calculating statistics
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

In [404]:
# run the selected t-test
stats.ttest_ind(a= men.values,
                b= women.values,
                equal_var=False)

Ttest_indResult(statistic=1.3437005678090845, pvalue=0.18785555340784144)

As a result, if we are using the threshold of 5% (which is a 95% confidence interval), then we cannot reject the null hypothesis. Also, since our female sample is rather small, we think it is reasonable to keep the threshold at least 5%. Hence, we can conclude that there is not enough evidence to suggest that the means for male and female students are not equal (at a 95% confidence level).

# Part 2

Similar to Part 1, we get the necessary data for the master students.

In [405]:
t = df.reset_index().drop("index",axis=1)
dfFM2 = t.set_index(['No Sciper','Semester'])

#these two can be written in one line, one var
#df.loc[df['column_name'] == some_value]

master_1=dfFM2.iloc[dfFM2.index.get_level_values(1).str.contains('Master semestre 1')].index.get_level_values(0)
master_2=dfFM2.iloc[dfFM2.index.get_level_values(1).str.contains('Master semestre 2')].index.get_level_values(0)
master_3=dfFM2.iloc[dfFM2.index.get_level_values(1).str.contains('Master semestre 3')].index.get_level_values(0)
hasMinor=dfFM2.loc[dfFM2["Mineur"].notnull()].index.get_level_values(0)
project=dfFM2.iloc[dfFM2.index.get_level_values(1).str.contains('Projet')].index.get_level_values(0)

#if master1 & master2 & project & not(Minor) or master1 & master2 & master3 & project & Minor
master=dfFM2.iloc[ (dfFM2.index.get_level_values(0).isin(master_1) 
                    & dfFM2.index.get_level_values(0).isin(master_2) 
                    & dfFM2.index.get_level_values(0).isin(project) 
                    & (dfFM2.index.get_level_values(0).isin(hasMinor) == False))
                | ( dfFM2.index.get_level_values(0).isin(master_1) 
                   & dfFM2.index.get_level_values(0).isin(master_3) 
                   & dfFM2.index.get_level_values(0).isin(master_2)
                   & dfFM2.index.get_level_values(0).isin(hasMinor) 
                   & dfFM2.index.get_level_values(0).isin(project))]
scipers=master.index.get_level_values(0)

In [406]:
filtered_mat = dfFM2.iloc[dfFM2.index.get_level_values(0).isin(scipers) & (dfFM2.index.get_level_values(1).str.contains('Master') | dfFM2.index.get_level_values(1).str.contains('Projet') )]
filtered_mat.reset_index(level =1, inplace=True)
filtered_mat

2,Semester,Civilité,Nom Prénom,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange
No Sciper,,,,,,,,,
114587,"Informatique, 2004-2005, Master semestre 1 (1...",Monsieur,Bonzon François,NaN,NaN,NaN,Présent,NaN,NaN
146753,"Informatique, 2004-2005, Master semestre 1 (1...",Monsieur,Schmid Antoine,NaN,NaN,NaN,Présent,NaN,NaN
114587,"Informatique, 2004-2005, Master semestre 2 (9...",Monsieur,Bonzon François,NaN,NaN,NaN,Présent,NaN,NaN
146753,"Informatique, 2004-2005, Master semestre 2 (9...",Monsieur,Schmid Antoine,NaN,NaN,NaN,Présent,NaN,NaN
153251,"Informatique, 2005-2006, Master semestre 1 (1...",Monsieur,Schild Stéphane,NaN,NaN,NaN,Présent,NaN,NaN
147545,"Informatique, 2005-2006, Master semestre 1 (1...",Monsieur,Tang Anli,NaN,NaN,NaN,Présent,NaN,NaN
153251,"Informatique, 2005-2006, Master semestre 2 (8...",Monsieur,Schild Stéphane,NaN,NaN,NaN,Présent,NaN,NaN
147545,"Informatique, 2005-2006, Master semestre 2 (8...",Monsieur,Tang Anli,NaN,NaN,NaN,Présent,NaN,NaN
114587,"Informatique, 2005-2006, Master semestre 3 (3...",Monsieur,Bonzon François,NaN,NaN,NaN,Présent,NaN,NaN


In [408]:
import re
def extractYear(x, level):
    pattern1 = re.compile('(20..)-.*')
    pattern2 = re.compile('-(20..).*')   
    if(level == 0):
        
        if(x.find("Bachelor semestre 1") >= 0):
            year = re.findall(pattern1, x)
        elif((x.find("semestre 6") >= 0)):
            year = re.findall(pattern2, x)
        elif((x.find("semestre 5") >= 0)):
            year = re.findall(pattern1, x)
        else: year = ['0']
        
    else:
        
        if(x.find("Master semestre 1") >= 0):
            year = re.findall(pattern1, x)
        elif((x.find("Projet") >= 0)):
            year = re.findall(pattern2, x)
        else: year = ['0']
    return int(year[0])

In [409]:
# Filter out the irrelevant semester

filtered_mat['Semester'] = filtered_mat['Semester'].map(lambda x : extractYear(x, 1))
filtered_mat = filtered_mat.loc[filtered_mat['Semester']!= 0]
filtered_mat= filtered_mat.set_index([filtered_mat.index.get_level_values(0), 'Semester'])
filtered_list= filtered_mat.groupby(filtered_mat.index.get_level_values(0)).apply(lambda x: list(x.index.get_level_values(1)))
months = pd.DataFrame(filtered_list)
months = months.rename(columns = {0:'Months'})
#fuck = fuck.apply(lambda x: x['years'].apply(computeMonths))
months
months = months['Months'].apply(computeMonths)

No Sciper
114587    24
145957    24
146742    42
146753    24
147545    24
153251    24
153819    24
154573    36
159516    24
159852    36
160150    24
160213    30
160225    24
161279    24
165819    24
166075    36
166344    24
166491    48
166805    24
170176    24
170235    36
170239    24
170265    24
171091    24
171195    24
171619    24
172264    24
172687    24
173257    24
173527    30
          ..
225323    24
225388    24
226037    24
227356    24
227848    36
228496    24
229235    36
229467    24
233184    36
233543    36
235878    36
240155    36
243163    24
243164    36
244136    36
244283    36
244587    36
245293    36
245486    36
245527    36
247455    36
254604    24
255137    24
255163    24
256722    24
256846    24
259766    24
259910    24
260636    24
260811    24
Name: Months, dtype: int64

In [411]:
months.mean()

29.274193548387096

Specialisation

In [422]:
spec = dfFM2.loc[dfFM2["Spécialisation"].notnull()].index.get_level_values(0)
spec

Index(['155396', '154958', '169626', '149101', '171229', '171794', '147545',
       '180072', '165819', '172611',
       ...
       '218353', '212859', '201600', '204253', '217301', '244587', '216702',
       '195675', '217469', '205864'],
      dtype='object', name='No Sciper', length=735)

In [425]:
specM = filtered_mat.iloc[filtered_mat.index.get_level_values(0).isin(spec)]
specM.reset_index(level =1, inplace=True)
specM

2,Semester,Civilité,Nom Prénom,Spécialisation,Filière opt.,Mineur,Statut,Type Echange,Ecole Echange
No Sciper,,,,,,,,,
147545,2005,Monsieur,Tang Anli,NaN,NaN,NaN,Présent,NaN,NaN
159852,2006,Monsieur,Brutsche Florian,NaN,NaN,NaN,Congé,Erasmus,"Royal Institute of Technology, (KTH) Stockholm"
160150,2006,Monsieur,Lépine Simon,NaN,NaN,NaN,Congé,Erasmus,University of Edinburgh
147545,2007,Monsieur,Tang Anli,Computer Engineering - SP,NaN,NaN,Présent,NaN,NaN
154573,2007,Madame,Benabdallah Zeineb,NaN,NaN,NaN,Présent,NaN,NaN
180072,2007,Monsieur,Campora Simone,Internet computing,NaN,NaN,Présent,NaN,NaN
165819,2007,Monsieur,Hofer Thomas,Foundations of Software,NaN,NaN,Présent,NaN,NaN
179878,2007,Monsieur,Muriel Hugo Marcelo,Internet computing,NaN,NaN,Présent,NaN,NaN
159852,2008,Monsieur,Brutsche Florian,Internet computing,NaN,NaN,Congé,NaN,NaN


In [427]:
specM= specM.set_index([specM.index.get_level_values(0), 'Semester'])
filtered_list2= specM.groupby(specM.index.get_level_values(0)).apply(lambda x: list(x.index.get_level_values(1)))
months2 = pd.DataFrame(filtered_list2)
months2 = months2.rename(columns = {0:'Months'})
months2 = months2['Months'].apply(computeMonths)

In [428]:
months2.mean()

33.096774193548384

In [430]:
master = months
specialization = months2

In [431]:
stats.ttest_ind(a= master.values,
                b= specialization.values,
                equal_var=False)

Ttest_indResult(statistic=-2.7314567589447898, pvalue=0.0090670855628446484)

Basically, with the 95% confidence we can reject null hypothesis, which is that the means are equal.